In [1]:
import nbimporter
import numpy as np
import matplotlib.pyplot as plt
import time
import moisture
import flux
import balance_equation
import property_conversion  as prop
import moisture_conversion as mc

Importing Jupyter notebook from moisture.ipynb
Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb
Importing Jupyter notebook from flux.ipynb
Importing Jupyter notebook from balance_equation.ipynb
Importing Jupyter notebook from property_conversion.ipynb


## 1次元モデルの差分計算をするためのモジュール  
目標：Cellを定義したらCell間の流量を計算してくれる  
→境界条件を入力したら境界の流量も計算してくれる  
→現在のCell情報と流量情報を入れたら新しい温度・水分状態を出してくれる  
これができるだけで室の温湿度計算などは簡単にできる

### クラス：Cell( i )  
差分法におけるコントロールボリューム（CV）に相当する構造体。  
固相・液相・気相の情報から構成され、CVの大きさ、位置などの情報を有する。  
示量変数はCellクラスに内包され、示強変数はそれぞれの相情報に保持される。

- cell情報  
    i：セルの位置    
    dx：セルの大きさ    


- 気相情報  
    - 水蒸気  
        temp：温度[K]  
        rh：相対湿度[-]  
        miu：水分化学ポテンシャル[J/kg]  
        pv：水蒸気圧[Pa]  
    - 空気（未実装）


- 液相情報  
    - 純水  
        temp：温度[K]  
        miu：水分化学ポテンシャル[J/kg]  
        phi：含水率[-]  
        pl：水圧[Pa]（未実装）  
    - 塩溶液（未実装）  
        

- 固相情報  
    - 材料骨格部  
        material：材料の名称  
        crow：熱容量（比熱×密度）  
        ※以下は液相・気相の影響を受ける  
        lam：熱伝達率  
        phi：含水率  
        dphi：含水率の水分化学ポテンシャル微分  
        ldp：透湿率  
        ldml：水分化学ポテンシャル勾配に対する液相水分伝導率  
    - 氷（未実装）   
    - 塩（未実装）  


In [2]:
class  Cell():
    def __init__( self, i ):
        self.position = i
    
    # cell information
    # Length of cell
    def setDx( self, initial = 0.0 ):
        self.dx = initial 
    
    # mass point(質点の位置)
    def setMassPoint( self, initial = 0.0 ):
        self.mp = initial
    
    # vapour condition（水蒸気の情報）
    def setVapour_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.vapour = moisture.MiuBasedMoisture( temp, miu )
        
    def setVapour_BasedRH( self, temp = 0.0, rh = 0.0 ):
        self.vapour = moisture.RHBasedMoisture( temp, rh )
        
    def setVapour_BasedPV( self, temp = 0.0, pv = 0.0 ):
        self.vapour = moisture.PVBasedMoisture( temp, pv )
        
    # Liquid condition（液水・溶液の情報）
    def setLiquidWater_Equilibrium( self ):
        self.water = self.vapour # 含水率などの情報も
        
    # physical property（材料情報）
    def setMaterial( self, name ):
        self.material = prop.MaterialKernel( name )
        
    def setMaterial_temp( self ):
        self.material.temp = self.vapour.temp
    

In [3]:
# 使用例
c = [ Cell(i) for i in range(10)]

[ c[i].setDx(0.01*i) for i in range(10)]
[ c[i].setMassPoint(0) for i in range(10)]
[ c[i].setVapour_BasedRH( temp = 293.15+i, rh = 0.6 ) for i in range(10) ]
[ c[i].setMaterial( 'BentheimerSandstone' ) for i in range( 10 )]
[ c[i].setLiquidWater_Equilibrium() for i in range(10) ]
[ c[i].setMaterial_temp() for i in range(10) ]

print(c[2].vapour.get_miu(), c[2].vapour.miu )

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from van_genuchten.ipynb
-69593.96317819103 -69593.96317819103


## 各種流量の計算  
前後のセルを与えることでセル間の流量を計算するだけのモジュール  
セルの質点から界面までの距離を計算するためmass pointという概念を導入した。  
mp = 0 なら質点はセルの中央  
mp = -1 なら質点はセルのマイナス側境界  
mp = 1 なら質点はセルのプラス側境界  
mns：前のセル情報  
pls：後のセル情報    

前のセルと後ろのセルを与えるだけで流量の計算をしてみる。

In [4]:
# 熱流計算
def flux_heat_v1( cell_mns, cell_pls ):
    if cell_mns.mp == 0:
        dx_mns = cell_mns.dx / 2.0
    elif cell_mns.mp == -1:
        dx_mns = cell_mns.dx
    elif cell_mns.mp == 1:
        dx_mns = 0.0
    if cell_pls.mp == 0:
        dx_pls = cell_pls.dx / 2.0
    elif cell_mns.mp == -1:
        dx_pls = cell_pls.dx
    elif cell_mns.mp == 1:
        dx_pls = 0.0
    
    return flux.heat_conduction_v2( lamL1 = cell_pls.material.getLAM(cell_pls.vapour), lamL = cell_mns.material.getLAM(cell_mns.vapour),\
                                   tempL1 = cell_pls.material.temp, tempL = cell_mns.material.temp, \
                                   dxL1   = dx_pls, dxL = dx_mns )

In [5]:
flux_heat_v1( c[1], c[2] )

-80.0

計算式が大きくなりすぎており見づらい。質点からセル境界までの距離は予め別の関数で計算させてみる。

In [6]:
def length_fromMassPoint( cell ):
    if cell.mp == 0:
        dx2 = cell.dx / 2.0
    elif cell.mp == -1:
        dx2 = cell.dx
    elif cell.mp == 1:
        dx2 = 0.0
    return dx2

In [7]:
def flux_heat_v2( cell_mns, cell_pls, dx2_mns, dx2_pls ):
    return flux.heat_conduction_v2( lamL1 = cell_pls.material.getLAM(cell_pls.vapour), lamL = cell_mns.material.getLAM(cell_mns.vapour),\
                                   tempL1 = cell_pls.material.temp, tempL = cell_mns.material.temp, \
                                   dxL1   = dx2_pls, dxL = dx2_mns )

In [8]:
dx2_mns = length_fromMassPoint( c[1] )
dx2_pls = length_fromMassPoint( c[2] )
flux_heat_v2( c[1], c[2], dx2_mns, dx2_pls )

-80.0

使いづらさが目立つ。クラス化することで整理してみる。

In [9]:
class Flux():
    def __init__( self, mns, pls ):
        self.mns = mns
        self.pls = pls
        
        # 質点間の距離 mns側の質点が右側境界にあること、その逆は基本的にない
        if self.mns.mp == -1:
            self.mns.dx2 = self.mns.dx
        else:
            self.mns.dx2 = self.mns.dx / 2.0
        if self.pls.mp == 1:
            self.pls.dx2 = self.pls.dx
        else :
            self.pls.dx2 = self.pls.dx / 2.0
        
    # 熱流計算
    def heat( self ):
        self.qs   = flux.heat_conduction_v2( lamL1 = self.pls.material.getLAM(self.pls.vapour), lamL = self.mns.material.getLAM(self.mns.vapour),\
                                   tempL1 = self.pls.material.temp, tempL = self.mns.material.temp, \
                                   dxL1   = self.pls.dx2, dxL = self.mns.dx2 )

In [10]:
# 使用例
cal_flux = [ Flux( c[i], c[i+1] ) for i in range ( 9 ) ]
[ cal_flux[i].heat() for i in range (9)]
[ cal_flux[i].qs for i in range( 9 ) ] 

[-240.0,
 -80.0,
 -47.99999999999999,
 -34.285714285714285,
 -26.666666666666664,
 -21.818181818181817,
 -18.46153846153846,
 -16.0,
 -14.117647058823529]

####  関数：calc( cell, boundary_start, boundary_end,  dt )
ここでは気相は水蒸気圧で、液相は水分化学ポテンシャル勾配で移動するとした

コンストラクタ（__init__）:  
差分計算用に温湿度・物性情報を差分化する  

Flux():  
セル間の流量を計算する
流量自体は「クラス名.q」や「クラス名.j」で取得できる  

balanceEquation():  
セルにおける熱・水分の収支計算を行う  

replace():
セルの温度・水分情報の更新を行う


In [ ]:
def calc( cell, boundary_start, boundary_end, dt ):

    # 物性値の取得 
    pro  = [ prop.PhysicalProperty( cell[i].material, cell[i].temp, cell[i].miu )  for i in range( len(cell) )]
    [ cell[i].setCRow( pro[i].CRow() ) for i in range( len(cell) )]
    [ cell[i].setDPhi( pro[i].DPhi() ) for i in range( len(cell) )]
    [ cell[i].setLAM(  pro[i].LAM()  ) for i in range( len(cell) )]
    [ cell[i].setLDP(  pro[i].DP()  ) for i in range( len(cell) )]
    [ cell[i].setLDML( pro[i].LDML() ) for i in range( len(cell) )]
    
    #########################
    # 流量計算
    flux = [ Flux( Length = len(cell), mns = cell[i], pls = cell[i+1], nx = 0.0 ) for i in range ( len(cell) - 1 ) ]
    [ flux[i].heat()   for i in range( len(cell)  -1 ) ]
    [ flux[i].vapour() for i in range( len(cell)  -1 ) ]
    [ flux[i].liquid() for i in range( len(cell)  -1 ) ]
        
    #########################
    # 収支計算
    newvalue = [ BalanceEquation.Calculation( cell[i], dt ) for i in range( len(cell) ) ]
    [ newvalue[i+1].differenceOfFlux( flux_mns = flux[i], flux_pls = flux[i+1] ) for i in range( len(cell) - 2 ) ]
        
    # 境界以外のセル
    [ newvalue[i+1].newTemp() for i in range( len(cell) - 2 ) ]
    [ newvalue[i+1].newMiu () for i in range( len(cell) - 2 ) ]
    
    # 境界セル
    newvalue[0].boundaryCell_Miu ( boundaryFlux = flux[0], boundary = boundary_start )
    newvalue[0].boundaryCell_temp( boundaryFlux = flux[0], boundary = boundary_start ) 
    
    newvalue[len(cell)-1].boundaryCell_Miu ( boundaryFlux = flux[len(cell)-2], boundary = boundary_end )
    newvalue[len(cell)-1].boundaryCell_temp( boundaryFlux = flux[len(cell)-2], boundary = boundary_end )

    #########################    
    # 値の換算
    [ cell[i].setTemp( newvalue[i].temp ) for i in range( len(cell) )]
    [ cell[i].setMiu ( newvalue[i].miu  ) for i in range( len(cell) )]
    [ cell[i].setPv  ( mc.cal_MiutoPv( cell[i].temp, cell[i].miu )  ) for i in range( len(cell) )]
    [ cell[i].setRH  ( mc.cal_MiutoRH( cell[i].temp, cell[i].miu )  ) for i in range( len(cell) )]
    

### 値を取得するクラス  
#### クラス : logger()  
ある点の温度をlogging間隔ごとに取得するクラス  

#### クラス：CrossSection()  
ある時刻における全セルの状態を取得するクラス  

In [ ]:
class CrossSection():
    def __init__( self, interval ):
        self.interval = interval
        self.times = []
        self.cell  = []
        
    def writeData( self, t, cell, dt):
        if t % self.interval == 0:
            self.times.append( t * dt)
            self.cell.append(cell)

### Draw Graph（グラフの描画をする関数）  

In [ ]:
# draw graph
def drawGraph( cell, time ):
    temp = [ cell[i].temp - 273.15 for i in range(L) ]
    dx   = [ cell[i].position*cell[i].dx for i in range(L) ]

    fig = plt.figure(figsize=(15,4))
    ax = fig.add_subplot(1,1,1)
    ax.plot( dx, temp, label = time )
    ax.set_xlabel( 'xlength[m]', fontsize = 12 )
    ax.set_ylabel( 'temp[K]', fontsize = 12 )
    plt.ylim([10.0, 20.0])
    plt.legend()
    plt.show()

### 計算  

#### 全体の流れ  

・壁体構成の入力（物性情報、セルの大きさ・個数、初期温湿度）  
・計算条件の入力（熱・水分の計算方法、境界条件）  
・計算に必要な物性値の決定

ループ計算内  

・物性値の取得→換算（差分型へ）  
・流量計算  
・収支計算→値の変換  
・値の取得（logging）  

終了　→　グラフ化など


In [ ]:
# セル情報のセッティング
L = 20
dt = 0.1

cell = [ Cell(i) for i in range( L ) ]

# セルの環境情報の入力
[ cell[i].setTemp( 288.15 ) for i in range( len(cell) ) ]
[ cell[i].setRH  ( 0.8 )    for i in range( len(cell) ) ]
[ cell[i].setMiu( mc.cal_RHtoMiu( cell[i].temp, cell[i].rh )) for i in range( len(cell) ) ]
[ cell[i].setPv(  mc.cal_RHtoPv(  cell[i].temp, cell[i].rh )) for i in range( len(cell) ) ]

# セル情報・環境情報の入力
[ cell[i].setMaterial( 'BentheimerSandstone' )for i in range( len(cell) ) ]
[ cell[i].setDx(0.001) for i in range( len(cell) )]

# initialtemp is the initial temperatures at the cells in the wall
# surfCond is the surface conductance at the both side surface of the wall (W/m2K)
# composition is the material of the cells in the wall
# material_porp is the material property list written in the dictionary style

# 境界条件
# 開始側境界条件
boundary_start = BalanceEquation.BoundaryCondition( 0 )
boundary_start.setHeatCondition( 'robin' ) 
boundary_start.setVapourCondition( 'robin' )
boundary_start.setLiquidCondition( 'neumann' )
boundary_start.setTemp( 283.15 )
boundary_start.setAL  ( 19.0 + 4.4 )
boundary_start.setPv  ( mc.cal_RHtoPv( boundary_start.temp, 0.85 ) )
boundary_start.setALDP( 19.0 / ( 1005.0 * 1.205 * ( 8314.41 / 18.02 ) * boundary_start.temp ) )
boundary_start.setJl  ( 0.0 )

# 終了側境界条件
boundary_end = BalanceEquation.BoundaryCondition( len(cell)-1 )
boundary_end.setHeatCondition( 'robin' ) 
boundary_end.setVapourCondition( 'robin' )
boundary_end.setLiquidCondition( 'neumann' )
boundary_end.setTemp( 293.15 )
boundary_end.setAL  ( 19.0 + 4.4 )
boundary_end.setPv  ( mc.cal_RHtoPv( boundary_start.temp, 0.7 ) )
boundary_end.setALDP( 19.0 / ( 1005.0 * 1.205 * ( 8314.41 / 18.02 ) * boundary_start.temp ) )
boundary_end.setJl  ( 0.0 )

# logging情報  
flux = [ Flux( L, cell[i], cell[i+1], 0.0 ) for i in range ( len(cell)-1 ) ]
log = CrossSection (60.0/dt)

print('Initialization')

In [ ]:
import time
start = time.time()
for i in range( 360000 ):
    # 境界条件はループの外に 
    calc( cell, boundary_start, boundary_end, dt )
    #log.writeData( i, cell, dt )
    if i % 3000 == 0 :
        print( i * dt / 60.0,"min")
        #print( cell[0].vol / cell[0].dx, cell[0].miulc, time.time()-start )
        drawGraph( cell, i*dt/60.0 )
        
print('time',time.time() - start)

In [ ]:
# draw graph
time = len(log.times)
c = log.cell[time-1]
temp = [ c[i].temp-273.15 for i in range(L) ]
dx =   [ c[i].position*c[i].dx for i in range(L) ]

fig = plt.figure(figsize=(15,4))
ax = fig.add_subplot(1,1,1)
ax.plot( dx, temp , label = log.times[time-1] )
ax.set_xlabel( 'xlength', fontsize = 12 )
ax.set_ylabel( 'temperature(K)', fontsize = 12 )
plt.legend()
plt.show()
